# A Deep-Learning Technique to Locate  Cryptographic Operation in Side-Channel Traces

In [ ]:
! pip install -r requirements.txt

In [1]:
import numpy as np
from tqdm.auto import tqdm

from CNN.train import train 
from inference_pipeline.sliding_window_classification import getModule, classifyTrace, saveClassification
from inference_pipeline.segmentation import squareWave, medianFilter, getStartCOs, plotStartCOs
from inference_pipeline.alignment import alignCos, saveCos

## Train CNN

Each CNN is configure thanks to a YALM configuration file.  
You can set different module hyper-parameters as well as the dataset, the logger, and the experiment configurations.  
Default configuration are in `CNN/configs` directory, both for Neputune logger and for the experiment. 

In [ ]:
config_folder = "CNN/configs/exp" # /path/to/experiment/config/folder/

In [ ]:
train(config_folder)

## Sliding Window Classification

In [ ]:
output_file = "</path/to/output_file>"
trace_file = "</path/to/trace_file>"
SID = "<Neptune_SID>"

Get the trained CNN starting from Neptune SID.

In [ ]:
module = getModule(SID)

Compute and save classification as file `output_file`.  
Function _classifyTrace_ has a few parameters to configure based on the experiment:
- `stride`: Define the stride to use for the sliding window.
- `window_size`: Define the size of the sliding window itself.

In [ ]:
stride = 1_000
window_size = 20_000

In [ ]:
classification = classifyTrace(trace_file, module, stride, window_size)
saveClassification(classification, output_file)

## Segmentation

Compute the segmenation and find the starting sample of each cryptografic operation.  
Functions _medianFilter_ and _getStartCOs_ have a few parameters to configure based on the experiment.

In [ ]:
classification_file = output_file
score_class = 1
min_distance = 200
kernel_size = 3

In [ ]:
classifications = np.load(classification_file, mmap_mode='r')[:, :, score_class]
startCOs = []

for classification in tqdm(classifications):
    square_wave = squareWave(classification)
    clean_square_wave = medianFilter(square_wave, kernel_size)
    startCOs.append(getStartCOs(clean_square_wave, min_distance, noise_applications=False))

n_COs = sum([len(co) for co in startCOs])
print(f"{n_COs} COs have been found in the trace.")

In [ ]:
plotStartCOs(startCOs, classifications)

## Alignment

Align in time the found COs.  
Some parameters, such as _slide_, has to be set as used in sliding window classification.

In [ ]:
trace_file =  "</path/to/trace_file>" # same trace as input of Sliding Window Classification
output_file = "</path/to/output_file>" 
stride = 1_000 # same value as input of Sliding Window Classification

In [ ]:
trace =  np.load(trace_file, mmap_mode='r')
COs = alignCos(trace, startCOs, stride)
saveCos(output_file, COs)